In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings

from __future__ import print_function

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (32, 32), cv2.INTER_LINEAR)
    return resized

def get_im_tf(path, img_rows, img_cols, color_type=1):
    img = tf.read_file(path)
    # Load as grayscale
    if color_type == 1:
        img = tf.image.decode_jpeg(img, channels=1)
    elif color_type == 3:
        img = tf.image.decode_jpeg(img, channels=0)
    # Reduce size
    resized = tf.image.resize_images(img, (32,32), method= 0,align_corners=False)
    return resized


def load_train(img_rows, img_cols, color_type=1):
    x_train = []
    x_train_id = []
    y_train = []
    #start_time = time.time()

    print('Read train images')
    folders = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('..', 'input', 'train', fld, '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl)
            x_train.append(img)
            x_train_id.append(flbase)
            y_train.append(index)

    #print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return x_train, y_train, x_train_id


def load_test():
    path = os.path.join('..', 'input', 'test_stg1', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id

def to_categorical(y, nb_classes=None):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to nb_classes).
        nb_classes: total number of classes.
    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int').ravel()
    if not nb_classes:
        nb_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, nb_classes))
    categorical[np.arange(n), y] = 1
    return categorical

In [ ]:
train_data, train_target, train_id = load_train(32,32,1)
test_data, test_id = load_test()


x_train = np.array(train_data, dtype=np.uint8)
y_train = np.array(train_target, dtype=np.uint8)
#x_train = x_train.transpose((0, 1, 2))

x_test = np.array(test_data, dtype=np.uint8)
#x_test = x_test.transpose((0, 3, 1, 2))

# Normalizing the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#x_train.reshape()
print(y_train.shape)
print('X_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_width = 32 # Fishes data input (img shape: 28*28)
n_height=32
n_classes = 8 # Fishes total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units


# tf Graph input
images_batch = tf.placeholder(tf.float32, [None,n_width,n_height,3])
labels_batch = tf.placeholder(tf.float32, [None, n_classes])
y_train = to_categorical(y_train, n_classes)
print(y_train.shape)
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [ ]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='VALID')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    print("Pablo0")
    #x = tf.reshape(x, shape=[-1, 32, 32, 3])
    
    print("Pablo1")

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(images_batch, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=labels_batch))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

print("prediction")
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(labels_batch, 1))
print("prediction")
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess: 
    sess.run(init) 
    for _ in range(1000):  
              
            indices = np.random.choice(3777, batch_size)
            print(indices.size)
            X_batch, Y_batch = x_train[indices], y_train[indices]
            print(X_batch.size)
            print(Y_batch)
            sess.run(optimizer, feed_dict={images_batch: X_batch, labels_batch: Y_batch,
                                       keep_prob: dropout})
            
            print(Y_batch.size)               
            if step % display_step == 0:
            # Calculate batch loss and accuracy
               loss, acc = sess.run([cost, accuracy], feed_dict={images_batch: X_batch,
                                                              labels_batch: Y_batch,
                                                              keep_prob: 1.})     
            print(acc+ " ")  
            step += 1
    
    print("Optimization Finished!")
            
 


